# Single Region Example

The following example shows how to use the assetra package to analyze an existing energy system. California ISO is taken as a use-case with system-specific data sources.

### 1. Setup *assetra*

To begin, we initialize an `EnergySystemBuilder` object. We will add demand and generating units to this system builder in the following sections.

In [42]:
from assetra.system import EnergySystemBuilder

builder = EnergySystemBuilder()

### 2. Load Demand Data

In this example, we use publicly-available historical hourly demand profiles based on the EIA-930 dataset. 

**Source:** https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

In [43]:
from pathlib import Path
from datetime import datetime

import xarray as xr
import pandas as pd

def load_eia_930_cleaned_hourly_demand(
        eia_930_cleaned_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data. This function expects a formatted csv which can
    be downloaded from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data
    """
    # read demand file
    eia_930_df = pd.read_csv(
        eia_930_cleaned_demand_file,
        usecols=["date_time", "cleaned demand (MW)"],
        index_col="date_time",
        parse_dates=True,
    )

    # keep demand
    hourly_demand_pd = eia_930_df["cleaned demand (MW)"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    hourly_demand = xr.DataArray.from_series(hourly_demand_pd)
    return hourly_demand

eia_930_cleaned_demand_file = Path('sample_data', 'CISO.csv')
hourly_demand = load_eia_930_cleaned_hourly_demand(
	eia_930_cleaned_demand_file,
	start_hour='2019-01-01 00:00:00',
	end_hour='2019-12-31 23:00:00'
)

The previous code block returns an hourly demand profile in an `xarray.DataArray` object. We can add this profile to our energy system as an `assetra.units.DemandUnit` object. 

**Notes:**

 - All objects inhering from `assetra.units.EnergyUnit` (including demand units) must have unique identifiying number. For the purposes of this example, we use an incrementing counter for this purpose.

In [44]:
from assetra.units import DemandUnit

# start unit count
unit_count = 0
builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)


### 2. Load Generator Data

We get unit-level generator data from EIA form-860. From this dataset, we identify nameplate capacity, location, and technology.

**Source:** https://www.eia.gov/electricity/data/eia860/

In [48]:
def load_eia_860_plants(eia_860_plant_file: Path, bal_auth: str) -> tuple[pd.Series, pd.Series, pd.Series]:
    """Return series of eia 860 plant codes, latitudes, and longitudes"""
    # read file
    eia_860_plant_df = pd.read_excel(
        eia_860_plant_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Latitude",
            "Longitude",
            "Balancing Authority Code",
        ],
        index_col="Plant Code",
    )

    # filter
    eia_860_plant_df = eia_860_plant_df[
        eia_860_plant_df["Balancing Authority Code"] == bal_auth
    ]

    return eia_860_plant_df

# parse eia 860 plants (selecting by balancing authority)
eia_860_plant_file = Path('sample_data', '2___Plant_Y2019.xlsx')
eia_860_plants = load_eia_860_plants(eia_860_plant_file, 'CISO')

def load_eia_860_generators(
    eia_860_generator_file: Path,
    plants: pd.DataFrame,
    additional_cols: list=[],
    tech_filter: list=[],
    invert_tech_filter: bool=False
    ) -> tuple[list, list, list]:
    """Return a list of thermal generator capacities, latitudes, and longitudes"""
    # read file
    eia_860_generator_df = pd.read_excel(
        eia_860_generator_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Technology",
            "Nameplate Capacity (MW)",
            "Status"
        ] + additional_cols,
    )

    # filter by plants
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Plant Code"].isin(plants.index)
    ]

    # filter by technology
    if tech_filter:
        if invert_tech_filter:
            eia_860_generator_df = eia_860_generator_df[
                ~eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]
        else:
            eia_860_generator_df = eia_860_generator_df[
                eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]

    # filter by status
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Status"] == "OP"
    ]

    return eia_860_generator_df

# parse eia 860 generator types
EIA_860_NON_THERMAL_TECHNOLOGY = [
    "Onshore Wind Turbine",
    "Conventional Hydroelectric",
    "Solar Photovoltaic",
    "Offshore Wind Turbine",
    "Batteries",
    "Hydroelectric Pumped Storage"
]
eia_860_generator_file = Path('sample_data', '3_1_Generator_Y2019.xlsx')
eia_860_wind_file = Path('sample_data', '3_2_Wind_Y2019.xlsx')
eia_860_solar_file = Path('sample_data', '3_3_Solar_Y2019.xlsx')
eia_860_storage_file = Path('sample_data', '3_4_Energy_Storage_Y2019.xlsx')

eia_860_thermal_generators = load_eia_860_generators(
    eia_860_generator_file, 
    eia_860_plants,
    tech_filter=EIA_860_NON_THERMAL_TECHNOLOGY,
    invert_tech_filter=True
)
eia_860_wind_generators = load_eia_860_generators(
    eia_860_wind_file,
    eia_860_plants
)
eia_860_solar_generators = load_eia_860_generators(
    eia_860_solar_file,
    eia_860_plants
)
eia_860_storage_generators = load_eia_860_generators(
    eia_860_storage_file,
    eia_860_plants,
    additional_cols=["Nameplate Energy Capacity (MWh)"]
)

We also need hourly solar and wind capacity factors and temperature profiles. We use another ASSET lab tool to generate sample data for CISO.

**Source:** https://github.com/ijbd/merra-power-generation


In [ ]:


# load processed power generation file created with 
pow_gen_file = 'merra_power_generation_ciso_2020.nc'
pow_gen_dataset = xr.open_dataset(pow_gen_file)

temp_dataset = pow_gen_dataset['T2M']
solar_c






In [ ]:
from assetra.preprocessing import (
	load_processed_reanalysis_net_cdf,
	loc_processed_reanalysis_variable,
	load_eia_860_plants,
	load_eia_860_thermal_generators,
	calc_hourly_forced_outage_rates
	)

generator_fleet = []

for generator in eia_860_thermal_generators:
	hourly_temperature = loc_processed_reanalysis_variable(
		variable_name='temperature (C)',
		start=datetime(2020, 1, 1),
		end=datetime(2021, 1, 1),
		latitude=generator['latitute']
		longitude=generator['longitude']
		)
	hourly_forced_outage_rate = calc_forced_outage_rate(
		technology=generator['technology'],
		temperature=hourly_temperature
		)

	generator_fleet.append(
		StochasticUnit(
			hourly_capacity=np.ones(8760)*generator['capacity (MW)']
			hourly_forced_outage_rate = hourly_forced_outage_rate
			)
		)

